---
title: "Data Cleaning"
format:
    html: 
        code-fold: false
---

## Introduction to Data Cleaning

Data cleaning is a critical step in ensuring the quality and usability of data for analysis. In this project, the data cleaning process involved merging data frames, creating more-indepth columns, handling missing variables, and coverting data types.

Since we want to analyze artists, songs, and lyrics, we conducted several changes such as adding song ranking, one hot encoding the genres, conducting sentiment analysis on the lyrics, and looking into artists features in a song. These will help streamline the EDA process as well as our analysis.


# Code 

Provide the source code used for this section of the project here.

If you're using a package for code organization, you can import it at this point. However, make sure that the **actual workflow steps**—including data processing, analysis, and other key tasks—are conducted and clearly demonstrated on this page. The goal is to show the technical flow of your project, highlighting how the code is executed to achieve your results.

If relevant, link to additional documentation or external references that explain any complex components. This section should give readers a clear view of how the project is implemented from a technical perspective.

Remember, this page is a technical narrative, NOT just a notebook with a collection of code cells, include in-line Prose, to describe what is going on.

In [466]:
import pandas as pd

In [467]:
#Read in all the raw-data
artists_data = pd.read_csv('../../data/raw-data/artists_data.csv')
rolling_stone = pd.read_csv('../../data/raw-data/rolling_stone_top_100.csv')
songs_lyrics = pd.read_csv('../../data/raw-data/songs_lyrics.csv')
tracks_data = pd.read_csv('../../data/raw-data/tracks_data.csv')

In [468]:
#The rolling_stone dataset has all the songs from rank 1-100 but tracks_data has two songs missing due to it not being found in the collection stage.
#We are creating blank rows so that we can input the song rank column into the tracks_data
empty_row = pd.Series(dtype='object') 

tracks_data = pd.concat(
    [tracks_data.iloc[:72], empty_row.to_frame().T, tracks_data.iloc[72:]],
    ignore_index=True
)

tracks_data = pd.concat(
    [tracks_data.iloc[:74], empty_row.to_frame().T, tracks_data.iloc[74:]],
    ignore_index=True
)

#We add the song rank into tracks_data
tracks_data['Song Rank'] = range(1, 101)
#Drop the empty columns as they will not be used. We just made them initially so that the ranking would not be wrong in tracks_data
tracks_data = tracks_data.dropna().reset_index(drop=True)

In [469]:
#Rename columns
artists_data = artists_data.rename(columns={'Name': 'Artists'})
tracks_data = tracks_data.rename(columns={'Popularity': 'Song Popularity'})
tracks_data = tracks_data.rename(columns={'Release Date': 'Song Release Date'})
tracks_data = tracks_data.rename(columns={'Artist': 'Artists'})
artists_data = artists_data.rename(columns={'Release Date': 'Album Release Date'})

#Merge column so we get artist and song info into one dataframe
artists_tracks_data = pd.merge(tracks_data, artists_data, on='Artists', how='left')

In [470]:
#We want to see how many artists were features on a song 
songs_lyrics['Total Artists On Song'] = songs_lyrics['Artists'].apply(lambda x: len(str(x).split('&')))

In [471]:
#Merge the lyrics dataframe with the artist/song masterlist dataframe
artists_tracks_data = pd.merge(artists_tracks_data, songs_lyrics, on='Track Name', how='left')

In [472]:
#Rename and drop redundant columns
artists_tracks_data = artists_tracks_data.drop(columns = ['Artists_y'])
artists_tracks_data = artists_tracks_data.rename(columns={'Artists_x': 'Artists'})
artists_tracks_data = artists_tracks_data.rename(columns={'lyrics': 'Lyrics'})

In [473]:
#Get the length of the lyrics 
artists_tracks_data['Lyrics Word Count'] = artists_tracks_data['Lyrics'].apply(lambda x: len(str(x).split()))

In [474]:
#Conducting sentimental analysis through VADER.
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

#Created a function reads in the lyrics through VADER
def analyze_sentiment(text):
    sentiment = analyzer.polarity_scores(str(text))
    return sentiment

artists_tracks_data['Sentiment (VADER)'] = artists_tracks_data['Lyrics'].apply(analyze_sentiment)

In [475]:
#Conducting TF-IDF on lyrics
from sklearn.feature_extraction.text import TfidfVectorizer

#Initialize/tokenize 
tfidf_vectorizer = TfidfVectorizer(
    #Remove english stop words
    stop_words='english', 
    #We want to just look at single words 
    ngram_range=(1, 1)     
)

tfidf_matrix = tfidf_vectorizer.fit_transform(artists_tracks_data['Lyrics'].fillna(''))

#Convert TF-IDF into a dataframe
tfidf_df = pd.DataFrame(
    tfidf_matrix.toarray(),
    #Use TF-IDF feature as column headers
    columns=tfidf_vectorizer.get_feature_names_out(),
    index=artists_tracks_data.index
)
tfidf_df.to_csv('../../data/processed-data/tfidf.csv')

In [476]:
from sklearn.preprocessing import MultiLabelBinarizer
artists_tracks_data['Genres'] = artists_tracks_data['Genres'].fillna('').astype(str)
# plit genres into a list as there could be multiple generes in a single cell 
artists_tracks_data['Genres Split'] = artists_tracks_data['Genres'].apply(lambda x: [genre.strip() for genre in x.split(',')])

mlb = MultiLabelBinarizer()

#Fit the mlb to the split genres
mlb.fit(artists_tracks_data['Genres Split'])

#Create a list of column names based on the genres
genre_columns = []
for col in mlb.classes_:
    genre_columns.append(f"Genre_{col}")

#One hot encode on the seperated genres
genre_one_hot = pd.DataFrame(
    mlb.fit_transform(artists_tracks_data['Genres Split']),
    columns=genre_columns,  
    index=artists_tracks_data.index
)

#Concatenate one hot encoded columns into artists_tracks_data 
artists_tracks_data = pd.concat([artists_tracks_data, genre_one_hot], axis=1)
artists_tracks_data = artists_tracks_data.drop(columns=['Genres Split'])


In [ ]:
artists_tracks_data.to_csv('../../data/processed-data/artist_song_masterlist.csv')